In [95]:
import json
import re
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import collections
import itertools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score

In [92]:
# Load the data
f = open("TVs-all-merged.json")
data = json.load(f)

# This function loops over the dictionary and the key value pairs
def pairs_dict(dict_object):
    for key, value in dict_object.items():
        if isinstance(value, dict):
            for product in pairs_dict(value):
                yield (key, *product)
        else:
            yield (key, value)


# Extracts the title of every product
counts = {}
All_titles = []
validation_df = pd.DataFrame(columns=['model ID', 'title'])
for pair in pairs_dict(data):
    for item in range(len(pair[1])):
        title = pair[1][item]['title']

        # Remove modelIDs from titles
        model_id = pair[0]
        if model_id in title:
            title = title.replace(model_id, '')

        # Remove al non-alphanumeric characters and standardize words
        title = re.sub('(/ |- |Newegg.com| TheNerds.net|Best Buy|,|\+|:|\)|\(| tv| TV|Class|;|*|$| )', '', title)
        title = re.sub('(-inch|inch|"|Inch|inches| inch| Inch|-Inch)', 'inch', title)
        title = re.sub('(Hz|hz|Hertz|hertz|HZ| hz|-hz| Hz)', 'hz', title)

        # Count the number of words in the titles
        for word in title.split():
            word = word.lower()
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
        All_titles.append(title.lower())

        validation_df = validation_df.append({'model ID': model_id, 'title': title}, ignore_index=True)
        
     


In [8]:
duplicates_number = 0
valuecount = validation_df['model ID'].value_counts()
for modelcount in valuecount:
    if modelcount == 2:
        duplicate_pairs += 1
    elif modelcount == 3:
        duplicate_pairs += 3
    elif modelcount == 4:
        duplicate_pairs += 6
    else:
        continue
print(duplicates_number)

# Sort counts and create a new dictionary with words that only appear once
wordcounts_one = {k: v for k, v in counts.items() if v == 1}
wordcounts = {k: v for k, v in counts.items() if v != 1}

# Remove words with single appearance
titles_final = []
for title in titles:
    for word1 in wordcounts_one.keys():
        if word1 in title.split():
            title = title.replace(word1, '')
    titles_final.append(title)

# data cleaning, removing the spaces at beginning and end
count = 0
for title in titles_final:
    if title.startswith(' '):
        title = title[1:]
    if title.startswith(' '):
        title = title[1:]
    if title.endswith(' '):
        title = title[:-1]
    if title.endswith(' '):
        title = title[:-1]
    titles_final[count] = title
    count += 1

# seperating each word and putting in seperate list
titles_split = []
for title in titles_final:
    titles_split.append(title.split())

# Make Boolean Matrix
mlb = MultiLabelBinarizer()
df = pd.DataFrame({"titles": titles_split})
df = pd.DataFrame(mlb.fit_transform(df['titles']), columns=mlb.classes_, index=df.index)
df = df.transpose()
deleted_titles = df.loc[:, (df == 0).all(axis=0)]
df = df.loc[:, (df != 0).any(axis=0)]
df = df.transpose()
df.reset_index(drop=True, inplace=True)
df = df.transpose()
for title in deleted_titles:
    validation_df = validation_df[validation_df.index != title]
validation_df.reset_index(drop=True, inplace=True)

399


In [79]:
count = 0
for model_id in 
    if validation_df[model_id] == 
print(validation_df)


            model ID                                              title
0       29PFL4508/F7  Philips 4000 Series 29inch 2812inch Diag. LED ...
1            SC-3211                   SuperSonic 32inch 720p LED HDTV 
2        LC-90LE657U  Sharp AQUOS 90inch 90inch Diag. LED 1080p 120h...
3       39PFL2908/F7  Philips 2000 Series 39inch 3858inch Diag. LED ...
4         LC70LE550U   Sharp Aquos 70inch 69.5inch Diagonal 1080p 12...
...              ...                                                ...
1619      TC-P55GT31  Panasonic VIERA 55inch  Plasma 1080p 600hz 3D ...
1620  UN40EH5000FXZA  Samsung 40inch 40inch Diag. LED 1080p 60hz HDTV  
1621  UN40EH5000FXZA  Samsung 40inch 40.0inch Diag. 1080p 60hz LED H...
1622    NS-32E859A11    Insignia Connected 32inch 1080p 120hz LED HDTV
1623            E424  NEC 42inch 42inch Diag. LEDLCD 1080p HDTV 1080p  

[1624 rows x 2 columns]


In [10]:
def intersection_list(lst_1, lst_2):
    list_intersection = [value for value in range(len(lst_1)) if lst_2[value] == 1 and lst_1[value] == 1]
    return list_intersection


def union_list(lst_1, lst_2):
    try:
        list_union = lst_1.value_counts()[1] + lst_2.value_counts()[1] - len(intersection_list(lst_1, lst_2))
    except:
        print(lst_1.value_counts())
        print(lst_2.value_counts())
        print(lst_1.value_counts()[1])
        print(lst_2.value_counts()[1])
    return list_union


def jaccardSim(d1, d2):
    return len(intersection_list(d1, d2)) / union_list(d1, d2)



In [88]:
signature_matrix = np.full((len(df.columns), 150), np.inf)
hash_functions = []
#Set seed
random.seed(123)
# generate the signature matrix using minhashing
for row in tqdm(range(len(df))):
    hash_row = []
    for i in range(150):
        np.random.seed(1)
        int1 = random.randint(0, 100)
        np.random.seed(2)
        int2 = random.randint(0, 100)
        hash_value = (int1 + int2 * (row + 1)) % 887
        hash_row.append(hash_value)
    hash_functions.append(hash_row)
    for column in df.columns:
        if (df.iloc[row][column] == 1):
            for i in range(len(hash_functions[row])):
                value = hash_functions[row][i]
                if value < signature_matrix[column][i]:
                    signature_matrix[column][i] = value
signmatrix = signature_matrix.transpose()


# Perform Locality Sensitive Hashing algporithm
b=30
r=5
n, d = signmatrix.shape
assert(n==b*r)
hash_buckets = collections.defaultdict(set)
bands = np.array_split(signmatrix, b, axis=0)
for k,band in enumerate(bands):
    for j in range(d):
        band_id = tuple(list(band[:,j])+[str(k)])
        hash_buckets[band_id].add(j)
candidate_pairs = set()
for bucket in hash_buckets.values():
    if len(bucket) > 1:
        for pair in itertools.combinations(bucket, 2):
            candidate_pairs.add(pair)

100%|██████████| 337/337 [02:25<00:00,  2.31it/s]


TypeError: 'tuple' object is not callable

In [87]:
# treshold t
lsh_pairs = set()
threshold = (1 / b) ** (1 / r)
for tuple in candidate_pairs:
    if jaccardSim(df[tuple[0]], df[tuple[1]]) > threshold:
        lsh_pairs.add((tuple[0], tuple[1]))
#print(lsh_pairs)
print(len(lsh_pairs))


# checking the number of real duplicates foun
classification_df = pd.DataFrame(columns=['candidates', 'duplicate label'])
for lshpair in lsh_pairs:
    array1 = df[lshpair[0]].to_list()
    array2 = df[lshpair[1]].to_list()
    array = array1 + array2
    if validation_df['model ID'][lshpair[0]] == validation_df['model ID'][lshpair[1]]:
        classification_df = classification_df.append({'candidates': array, 'duplicate label': 1}, ignore_index=True)
    else:
        classification_df = classification_df.append({'candidates': array, 'duplicate label': 0}, ignore_index=True)
classification_df.to_csv('class_df.csv')

# Pair completness and Pair Quality
found_duplicates = print(sum(classification_df['duplicate label']))
number_of_duplicates = 399
PQ = found_duplicates/number_of_duplicates


9723
28


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [102]:

# X_try = pd.DataFrame(columns=range(670))
# classi = pd.DataFrame(classification_df)
# classi = classi.drop(['duplicate label'])
# print(classi)
# for index, lst in enumerate(classification_df.candidates):
#         X_try[index] = lst
#         X_try[lst(index)] = classi[index].loc[lst]
Y = pd.Series(classification_df['duplicate label'].astype('int'))




X = pd.read_csv("X.csv", sep=",")
X = pd.DataFrame(X)

#Calculate PC
comparisons = nrow(X)
Pair_completness = found_duplicates/comparisons

predictions = []
f1_score = []

#Bootstrapping the Random Forest Algorithm
for bs in range(10):
    X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.37)
    ET = ExtraTreesClassifier(random_state=42, bootstrap=True).fit(X_train, Y_train)
    ypred = ET.predict(X_test)
    score = f1_score(Y_test, y_pred)
    f1_score.append(score)

finalprediction = np.mean(predictions)
print(finalpredicton)



print(X)

       Unnamed: 0  0  1  2  3  4  5  6  7  8  ...  660  661  662  663  664  \
0               0  0  0  1  0  0  0  0  0  0  ...    0    0    0    0    0   
1               1  0  0  1  0  1  0  0  0  0  ...    0    0    0    0    0   
2               2  0  0  1  0  1  0  0  0  0  ...    0    0    0    0    0   
3               3  0  0  1  0  1  0  0  0  0  ...    0    0    0    0    0   
4               4  0  0  1  0  1  0  0  0  0  ...    0    0    0    0    0   
...           ... .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...   
24105       24105  0  0  1  0  0  0  0  0  0  ...    0    0    0    0    0   
24106       24106  0  0  1  0  0  0  0  0  0  ...    0    0    0    0    0   
24107       24107  0  0  1  0  1  0  0  0  0  ...    0    0    0    0    0   
24108       24108  0  0  1  0  1  0  0  0  0  ...    0    0    0    0    0   
24109       24109  0  0  1  0  0  0  0  0  0  ...    0    0    0    0    0   

       665  666  667  668  669  
0        0    0    0    0    0